In [ ]:
def extract_and_filter_facets(input_path, output_path, z_threshold):
    with open(input_path, 'r') as infile:
        lines = infile.readlines()
    output_header = None
    filtered_facets = []
    current_facet = []
    record_facet = False
    for line in lines:
        stripped = line.strip()
        if stripped.startswith("solid") and output_header is None:
            output_header = line
            continue
        if stripped.startswith("facet normal"):
            current_facet = [line]
            parts = stripped.split()
            try:
                normal_z = float(parts[-1])
                record_facet = (normal_z < 0.0)
            except ValueError:
                record_facet = False
            continue
        if current_facet is not None:
            current_facet.append(line)
            if stripped == "endfacet":
                if record_facet:
                    z_vals = []
                    for facet_line in current_facet:
                        if facet_line.strip().startswith("vertex"):
                            try:
                                z_vals.append(float(facet_line.strip().split()[-1]))
                            except ValueError:
                                pass
                    if z_vals and all(z <= z_threshold for z in z_vals):
                        filtered_facets.extend(current_facet)
                current_facet = []
                record_facet = False
    with open(output_path, 'w') as outfile:
        outfile.write(output_header)
        for facet_line in filtered_facets:
            outfile.write(facet_line)
        outfile.write("endsolid\n")
    n_triangles = len(filtered_facets) // 7
    print(f"✅ Done: {n_triangles} facets with negative-Z normals and Z ≤ {z_threshold} saved to:\n  {output_path}")

if __name__ == "__main__":
    input_stl = r"Put Path of CAD Full.stl"
    output_stl = r"Put Path of CAD Outer.stl"
    threshold_value = 'Put height above which there should be no stl triangles'
    extract_and_filter_facets(input_stl, output_stl, threshold_value)
